In [1]:
kernel_mode = False

import os
import numpy as np
import pandas as pd
import time
import random
import math
import glob

import optuna

import warnings
warnings.filterwarnings('ignore')

import gc
gc.enable()

rand_seed = 1120

optuna_mode = False
study_name = "moa_blend_effnetv7b3_resnestv1v2"
n_trials = 500
# n_trials = 3000

In [2]:
dataset_folder = "../input/lish-moa" if kernel_mode else "/workspace/Kaggle/MoA/"

# Add your model inference script here
# Tuple Format: (script, oof_filename, output_filename, weight)
model_list = [
    ("deepinsight_efficientnet_lightning_v7_b3_infer.py",
     "oof_deepinsight_efficientnet_v7_b3_0.014802440208660929.npy",
     "submission_effnet_v7_b3.csv", 0.3287684605023437),
    
    ("deepinsight_resnest_lightning_v1_infer.py",
     "oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy",
     "submission_resnest_v1.csv", 0.2763485706536088),
    
    ("deepinsight_resnest_lightning_v2_infer.py",
     "oof_deepinsight_ResNeSt_v2_resnest50_0.01455961217985703.npy",
     "submission_resnest_v2.csv", 0.3859487453003219),
]

model_path = "../input" if kernel_mode else f"{dataset_folder}/completed"

In [3]:
train_labels = pd.read_csv(f'{dataset_folder}/train_targets_scored.csv',
                           engine='c')
train_classes = [c for c in train_labels.columns if c != "sig_id"]

submission = pd.read_csv(f'{dataset_folder}/sample_submission.csv')
submission.iloc[:, 1:] = 0

In [4]:
def mean_logloss(y_pred, y_true):
    logloss = (1 - y_true) * np.log(1 - y_pred +
                                    1e-15) + y_true * np.log(y_pred + 1e-15)
    return np.mean(-logloss)

In [5]:
total_start = time.time()
if not optuna_mode:
    for i, (script, oof_filename, output_filename, weight) in enumerate(model_list):
        print(f"Generating submission file from {script} ......")
        infer_start = time.time()
        !python {script}
        infer_elapsed = time.time() - infer_start
        print(f"Time spent on inference: {infer_elapsed/60:.2f} minutes.")

        model_submit = pd.read_csv(output_filename, engine='c')
        print(model_submit.head(5))
        print(model_submit.shape)
        submission.iloc[:, 1:] += weight * model_submit.iloc[:, 1:]
else:
    ## Search Best Blend Weights by Optuna ##
    model_oofs = []

    for i, (script, oof_filename, output_filename, weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")
        oof_loss = mean_logloss(oof, train_labels[train_classes].values)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}")
        model_oofs.append(oof)

    def objective(trial):
        weights = []
        for i in range(len(model_list)):
            weights.append(trial.suggest_float(f"w{i}", 0, 1.0))

        blend = np.zeros(model_oofs[0].shape)
        for i in range(len(model_list)):
            blend += weights[i] * model_oofs[i]
        blend = np.clip(blend, 0, 1.0)

        loss = mean_logloss(blend, train_labels[train_classes].values)
        return loss

    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=0,
        interval_steps=1,
    )
    sampler = optuna.samplers.TPESampler(seed=rand_seed)
    study = optuna.create_study(direction="minimize",
                                pruner=pruner,
                                sampler=sampler,
                                study_name=study_name,
                                storage=f'sqlite:///{study_name}.db',
                                load_if_exists=True)

    study.optimize(objective,
                   n_trials=n_trials,
                   timeout=None,
                   gc_after_trial=True,
                   n_jobs=-1)

    trial = study.best_trial

    for i, (script, oof_filename, output_filename, _) in enumerate(model_list):
        optimal_weight = trial.params[f"w{i}"]
        print(f"Generating submission file from {script} ...... (Weight: {optimal_weight})")
        infer_start = time.time()
        !python {script}
        infer_elapsed = time.time() - infer_start
        print(f"Time spent on inference: {infer_elapsed/60:.2f} minutes.")

        model_submit = pd.read_csv(output_filename, engine='c')
        print(model_submit.head(5))
        print(model_submit.shape)
        submission.iloc[:, 1:] += optimal_weight * model_submit.iloc[:, 1:]

    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

Loading OOF from oof_deepinsight_efficientnet_v7_b3_0.014802440208660929.npy ......
OOF Validation Loss of deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.014802
Loading OOF from oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy ......
OOF Validation Loss of deepinsight_resnest_lightning_v1_infer.py: 0.014620
Loading OOF from oof_deepinsight_ResNeSt_v2_resnest50_0.01455961217985703.npy ......


[I 2020-11-22 17:45:48,852] A new study created in RDB with name: moa_blend_effnetv7b3_resnestv1v2


OOF Validation Loss of deepinsight_resnest_lightning_v2_infer.py: 0.014560


[I 2020-11-22 17:45:49,726] Trial 1 finished with value: 0.020807341873103062 and parameters: {'w0': 0.6074139663418106, 'w1': 0.9869052331681507, 'w2': 0.8773250697995254}. Best is trial 1 with value: 0.015010924699453133.
[I 2020-11-22 17:45:49,771] Trial 4 finished with value: 0.01421170263886513 and parameters: {'w0': 0.34336312964292925, 'w1': 0.24376122013699886, 'w2': 0.29082954084322277}. Best is trial 4 with value: 0.01421170263886513.
[I 2020-11-22 17:45:49,828] Trial 0 finished with value: 0.015010924699453133 and parameters: {'w0': 0.930759429641148, 'w1': 0.04384276018231803, 'w2': 0.10059196497271616}. Best is trial 4 with value: 0.01421170263886513.
[I 2020-11-22 17:45:49,830] Trial 3 finished with value: 0.015939459225344833 and parameters: {'w0': 0.06363293969899397, 'w1': 0.8066950475368859, 'w2': 0.5021240920397887}. Best is trial 4 with value: 0.01421170263886513.
[I 2020-11-22 17:45:49,835] Trial 2 finished with value: 0.016064452585317086 and parameters: {'w0': 0.

[I 2020-11-22 17:45:54,125] Trial 36 finished with value: 0.014208027989423734 and parameters: {'w0': 0.18389440828622255, 'w1': 0.44914338751422134, 'w2': 0.3725936235232824}. Best is trial 32 with value: 0.014184874015211648.
[I 2020-11-22 17:45:54,128] Trial 34 finished with value: 0.014439137211864277 and parameters: {'w0': 0.00280097372438301, 'w1': 0.44498103337790457, 'w2': 0.3697205166603693}. Best is trial 32 with value: 0.014184874015211648.
[I 2020-11-22 17:45:54,130] Trial 37 finished with value: 0.014419355537113468 and parameters: {'w0': 0.16331485160588755, 'w1': 0.34543664959985887, 'w2': 0.5533845765731079}. Best is trial 32 with value: 0.014184874015211648.
[I 2020-11-22 17:45:54,400] Trial 38 finished with value: 0.014546794368630446 and parameters: {'w0': 0.049735906730630386, 'w1': 0.46027193489902907, 'w2': 0.5599357847215822}. Best is trial 32 with value: 0.014184874015211648.
[I 2020-11-22 17:45:54,413] Trial 39 finished with value: 0.014281279638778708 and para

...

Test iterations: 42
Testing: 100%|██████████████████████████████████| 42/42 [00:09<00:00,  5.26it/s]Logits: tensor([[ -7.5195,  -7.1797,  -5.7695,  ...,  -5.7656,  -5.8906,  -6.6289],
        [ -9.2266,  -8.9688,  -7.6719,  ...,  -7.7109, -12.4219,  -6.6680],
        [ -8.0469,  -8.3359,  -7.4805,  ...,  -7.5156,  -8.4531,  -7.3125],
        ...,
        [ -5.4414,  -7.2773,  -8.2266,  ...,  -7.5508,  -9.5469,  -6.4297],
        [ -7.6445,  -7.4336,  -6.3711,  ...,  -6.5156,  -9.1406,  -5.6562],
        [ -7.6172,  -6.6367,  -6.3750,  ...,  -5.7422,  -7.8867,  -6.3633]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[5.422e-04 7.615e-04 3.111e-03 ... 3.124e-03 2.758e-03 1.320e-03]
 [9.841e-05 1.273e-04 4.656e-04 ... 4.478e-04 4.053e-06 1.269e-03]
 [3.200e-04 2.397e-04 5.636e-04 ... 5.441e-04 2.131e-04 6.666e-04]
 ...
 [4.314e-03 6.905e-04 2.673e-04 ... 5.255e-04 7.141e-05 1.611e-03]
 [4.785e-04 5.908e-04 1.707e-03 ... 1.478e-03 1.072e-04 3.483e-03]
 [4.916e-04 1.309e-03 1

Generating submission file from deepinsight_resnest_lightning_v1_infer.py ...... (Weight: 0.2809444099420081)
PyTorch Version: 1.6.0+cu101
PyTorch Lightning Version: 1.0.4
Inferencing on Fold 0 ......
(21432,) (2382,)
Loading model from /workspace/Kaggle/MoA/completed/deepinsight_ResNeSt_v1_resnest50/deepinsight_ResNeSt_v1_resnest50/fold0/epoch25-train_loss_epoch0.016572-val_loss_epoch0.014525-image_size224-resolution224-perplexity5-fc512.ckpt
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Using native 16bit precision.
Test iterations: 16
Testing: 100%|██████████████████████████████████| 16/16 [00:13<00:00,  1.41it/s]Logits: tensor([[ -7.0039,  -7.3164,  -6.5586,  ...,  -6.7266,  -6.0352,  -5.7812],
        [-10.5312,  -8.0078,  -8.2500,  ...,  -7.3086, -10.0625,  -5.7656],
        [ -7.7461,  -7.5820,  -7.5547,  ...,  -7.6875,  -8.0078,  -7.4961],
        ...,
        [ -5.9141,  -6.7227,  -7.0273,  ...,  -6.9570,  

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Using native 16bit precision.
Test iterations: 16
Testing: 100%|██████████████████████████████████| 16/16 [00:13<00:00,  1.41it/s]Logits: tensor([[-7.5156, -6.6094, -6.1016,  ..., -6.6055, -5.6953, -6.1602],
        [-8.2031, -9.0078, -8.7109,  ..., -8.6094, -9.7188, -7.5508],
        [-7.5117, -7.5625, -7.2070,  ..., -7.6992, -7.6680, -7.4688],
        ...,
        [-5.1680, -5.4453, -7.6836,  ..., -7.3086, -8.1719, -6.2969],
        [-6.2930, -6.7188, -7.0078,  ..., -6.3281, -6.9883, -6.4062],
        [-6.2734, -7.4570, -6.4258,  ..., -6.9883, -8.2344, -6.8125]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[5.4407e-04 1.3456e-03 2.2335e-03 ... 1.3514e-03 3.3512e-03 2.1076e-03]
 [2.7370e-04 1.2243e-04 1.6475e-04 ... 1.8239e-04 6.0141e-05 5.2547e-04]
 [5.4646e-04 5.1928e-04 7.4100e-04 ... 4.5300e-04 4.6730e-04 5.7030e-04]
 ...
 [5.6648e-03 4.2992e-03 4.60

PyTorch Version: 1.6.0+cu101
PyTorch Lightning Version: 1.0.4
Inferencing on Fold 0 ......
(21432,) (2382,)
Loading model from /workspace/Kaggle/MoA/completed/deepinsight_ResNeSt_v2_resnest50/deepinsight_ResNeSt_v2_resnest50/fold0/epoch25-train_loss_epoch0.016863-val_loss_epoch0.014446-image_size224-resolution224-perplexity5-fc512.ckpt
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Using native 16bit precision.
Test iterations: 16
Testing: 100%|██████████████████████████████████| 16/16 [00:14<00:00,  1.40it/s]Logits: tensor([[-7.1992, -8.0703, -6.8320,  ..., -7.0938, -6.2148, -5.9062],
        [-9.7656, -9.1875, -7.5547,  ..., -7.7031, -9.1328, -5.5781],
        [-7.8945, -7.7383, -7.6133,  ..., -7.5938, -7.8750, -7.5664],
        ...,
        [-5.6328, -6.8633, -7.2227,  ..., -6.8398, -7.9336, -6.5352],
        [-7.1055, -7.6250, -7.3008,  ..., -6.4844, -7.0312, -5.6172],
        [-6.2266, -6.8242, -6.8398,  ..., -6

Inferencing on Fold 6 ......
(21433,) (2381,)
Loading model from /workspace/Kaggle/MoA/completed/deepinsight_ResNeSt_v2_resnest50/deepinsight_ResNeSt_v2_resnest50/fold6/epoch35-train_loss_epoch0.015504-val_loss_epoch0.014640-image_size224-resolution224-perplexity5-fc512.ckpt
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Using native 16bit precision.
Test iterations: 16
Testing: 100%|██████████████████████████████████| 16/16 [00:12<00:00,  1.41it/s]Logits: tensor([[-7.9414, -7.6016, -6.3906,  ..., -6.6133, -4.0391, -7.3008],
        [-9.5625, -9.4922, -6.1719,  ..., -8.0312, -7.8477, -6.4961],
        [-7.5938, -7.4297, -7.1289,  ..., -7.4531, -7.5352, -7.5977],
        ...,
        [-5.4102, -6.4961, -6.6406,  ..., -6.8711, -7.5547, -7.2305],
        [-6.8438, -7.1055, -7.7188,  ..., -6.8320, -7.0312, -6.2422],
        [-7.3789, -6.8438, -6.2344,  ..., -6.1641, -7.9336, -7.1406]],
       device='cuda:0', dtype=torch

In [6]:
total_elapsed = time.time() - total_start
print(f"Total time spent: {total_elapsed/60:.2f} minutes.")

Total time spent: 8.90 minutes.


In [7]:
# Number of finished trials: 500
# Best trial:
#   Value: 0.014158536219669974
#   Params: 
#     w0: 0.3287684605023437
#     w1: 0.2763485706536088
#     w2: 0.3859487453003219

In [8]:
# Number of finished trials: 3000
# Best trial:
#   Value: 0.014287989662394677
#   Params: 
#     w0: 0.4397214034464735
#     w1: 0.5466210401214696

In [9]:
print(submission.shape)
submission

(3982, 207)


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.001150,0.001175,0.001992,0.016708,0.028415,0.004133,0.003372,0.003285,0.000253,...,0.001509,0.000651,0.002634,0.001835,0.000815,0.000750,0.000764,0.002123,0.002089,0.001321
1,id_001897cda,0.000194,0.000272,0.000735,0.000335,0.000746,0.001457,0.002943,0.006195,0.057598,...,0.000326,0.000145,0.001738,0.000768,0.005100,0.000176,0.004178,0.000437,0.000190,0.002191
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000514,0.000415,0.001816,0.019875,0.016261,0.004196,0.003917,0.003938,0.000431,...,0.000819,0.001293,0.001764,0.011385,0.030854,0.000448,0.004340,0.002960,0.002097,0.003044
4,id_0027f1083,0.003664,0.001863,0.001500,0.015535,0.023608,0.004950,0.005192,0.001400,0.000333,...,0.001323,0.000743,0.002630,0.001146,0.001237,0.000874,0.001142,0.001485,0.000481,0.001666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,0.000261,0.000491,0.000595,0.002096,0.004209,0.000718,0.000628,0.001204,0.000387,...,0.000659,0.010083,0.002169,0.351550,0.007484,0.001036,0.004647,0.000505,0.000818,0.000354
3978,id_ff925dd0d,0.004772,0.003274,0.001054,0.010644,0.021122,0.006381,0.005981,0.002880,0.000274,...,0.000632,0.000638,0.003040,0.001608,0.001153,0.001079,0.002243,0.001833,0.000367,0.000897
3979,id_ffb710450,0.004154,0.002022,0.000944,0.009354,0.031037,0.009771,0.003335,0.001782,0.000250,...,0.000618,0.000697,0.002355,0.000973,0.001494,0.000768,0.000844,0.001164,0.000323,0.001666
3980,id_ffbb869f2,0.001590,0.000905,0.001136,0.027759,0.025187,0.003774,0.009341,0.002495,0.000660,...,0.000973,0.000555,0.002391,0.001240,0.001733,0.000497,0.000791,0.002367,0.000561,0.003013


In [10]:
submission.to_csv('submission.csv', index=False)